# Self Critique & Self Improvement

The pursuit of self-improving systems is driven by the promise of creating artificial intelligence that can learn, adapt, and evolve autonomously, leading to more efficient problem-solving and decision-making. Such systems could revolutionize a wide range of fields, from medicine and climate modeling to space exploration and poetry.

This notebook implements Noah Goodman's [meta-prompt algorithm](https://noahgoodman.substack.com/p/meta-prompt-a-simple-self-improving?subscribe_prompt=free)

In [ ]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], template=template
    )

    chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=prompt,
        verbose=True,
        memory=ConversationBufferWindowMemory(),
    )
    return chain


def initialize_meta_chain():
    meta_template = """
    Assistant has just had the below interactions with a User. 
    Assistant followed their "Instructions" closely. 
    Your job is to critique the Assistant's performance and 
    then revise the Instructions so that Assistant would 
    quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. 
    What could Assistant have done better? 
    What should the Assistant remember about this user? 
    Are there things this user always wants? 
    Indicate this with "Critique: ...".

    You should next revise the Instructions so that 
    Assistant would quickly and correctly respond in the future. 
    Assistant's goal is to satisfy the user in 
    as few interactions as possible. 
    Assistant will only see the new Instructions, 
    not the interaction history, so anything important must be 
    summarized in the Instructions. 
    Don't forget any important details in the current Instructions! 
    Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], template=meta_template
    )

    meta_chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=meta_prompt,
        verbose=True,
    )
    return meta_chain


def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history


def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter) :]
    return new_instructions

In [ ]:
def main(task, max_iters=3, max_meta_iters=5):
    failed_phrase = "task failed"
    success_phrase = "task succeeded"
    key_phrases = [success_phrase, failed_phrase]

    instructions = "None"
    for i in range(max_meta_iters):
        print(f"[Episode {i+1}/{max_meta_iters}]")
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f"(Step {j+1}/{max_iters})")
            print(f"Assistant: {output}")
            print(f"Human: ")
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print(f"You succeeded! Thanks for playing!")
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f"Feedback: {meta_output}")
        instructions = get_new_instructions(meta_output)
        print(f"New Instructions: {instructions}")
        print("\n" + "#" * 80 + "\n")
    print(f"You failed! Thanks for playing!")

In [4]:
task = "Provide a systematic argument for why we should always eat salad with dressing."
main(task)



> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: When the user asks for a systematic argument for why they should always eat salad with dressing, provide a response that outlines the health benefits of eating salad with dressing, such as increased nutrient absorption, improved flavor, and feeling fuller for longer. After providing the response, ask the user if they have any other questions about eating salad with dressing. If the user does not respond, ask if there is anything else the Assistant can help with.
    Human: Provide a systematic argument for why we should always eat salad with dressing.
AI:  Eating salad with dressing is beneficial for your health because it helps your body absorb more nutrients from the vegetables. Additionally, the dressing adds flavor to the salad, making it more enjoyable to eat. Eating salad with dressing can also help you feel fuller for longer, so you don't have to worry about snacking between meals. Do you have any oth